In [1]:
%pip install "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 2.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.2
    Uninstalling grpcio-1.54.2:
      Successfully uninstalled grpcio-1.54.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ray import tune

def objective(config):  # ①
    score = config["a"] ** 2 + config["b"]
    return {"score": score}


search_space = {  # ②
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    "b": tune.choice([1, 2, 3]),
}

tuner = tune.Tuner(objective, param_space=search_space)  # ③

results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

2023-06-30 19:04:00,160	INFO worker.py:1636 -- Started a local Ray instance.
2023-06-30 19:04:04,432	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


Trial name,date,done,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
objective_1e5c6_00002,2023-06-30_19-04-45,False,DESKTOP-UVK0SLT,1,172.27.82.99,5008,1.01,0.0314031,0.0314031,0.0314031,1688144685,1,1e5c6_00002


2023-06-30 19:04:46,436	INFO tune.py:1111 -- Total run time: 41.65 seconds (28.62 seconds for the tuning loop).


{'a': 0.001, 'b': 1}


In [1]:
%cd ..

/home/jakob/Bachelorarbeit/GNNDensityGradients


In [2]:
from pytorch_lightning.tuner import Tuner
from utils.visualization import fig_to_tensor
from glob import glob
import os.path
import pickle
import zstandard as zstd
import msgpack
import msgpack_numpy
msgpack_numpy.patch()

In [3]:
def transform_msgpack_file(filepath, transform):
    """
    Given a .zst file, applies the given transform to all stored simulation states and writes the result back to the file.
    """
    # TODO: Implement this to allow storing of preprocessed data
    # See https://github.com/isl-org/DeepLagrangianFluids/blob/d651c6fdf2aca3fac9abe3693b20981b191b4769/datasets/create_physics_records.py#L100

    decompressor = zstd.ZstdDecompressor()
    with open(filepath, 'rb') as f:
        content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

    for i in range(1, len(content)):
        content[i]['box'] = content[0]['box']
        content[i]['box_normals'] = content[0]['box_normals']

    content = [transform(sample) for sample in content]

    for i in range(1, len(content)):
        content[i]['box'] = None
        content[i]['box_normals'] = None

    compressor = zstd.ZstdCompressor(level=22)
    with open(filepath, 'wb') as f:
        f.write(compressor.compress(msgpack.packb(content, use_bin_type=True)))

    return

In [4]:
from utils.transforms import *
import torch
from utils.sph import *
import os
path = 'datasets/data/dpi_dam_break/train/test.zst'
os.path.isfile(path)

True

In [5]:
transformations = [
    ToSample(device='cpu'),
    AddDensity(include_box=False),
    AddTemporalDensityGradient(include_box=False),
    NormalizeDensityData(),
    ToNumpy()
]

In [9]:
from torchvision.transforms import Compose
transform = Compose(transformations)

transform_msgpack_file(path, transform)

In [10]:
filepath = path

decompressor = zstd.ZstdDecompressor()
with open(filepath, 'rb') as f:
    content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

content[1]

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals', 'density', 'temporal_density_gradient'])

In [14]:
content[0]['m']

array([[0.1308997],
       [0.1308997],
       [0.1308997],
       ...,
       [0.1308997],
       [0.1308997],
       [0.1308997]], dtype=float32)

In [19]:
from tqdm import tqdm
from time import sleep
for x in tqdm(["a", "b", "c"]):
    sleep(0.5)

100%|██████████| 3/3 [00:01<00:00,  1.96it/s]


In [22]:
from importlib import reload
import utils.train_helper
from utils.train_helper import *
reload(utils.train_helper); from utils.train_helper import *
prepocess_dataset_files('datasets/data/dpi_dam_break/train', type='temp_grad', include_box=False, device = 'cpu')

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:22<00:00,  1.85s/it]


In [23]:
filepath = path

decompressor = zstd.ZstdDecompressor()
with open(filepath, 'rb') as f:
    content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

content[1]['m']

array([[0.1308997],
       [0.1308997],
       [0.1308997],
       ...,
       [0.1308997],
       [0.1308997],
       [0.1308997]], dtype=float32)

In [1]:
%cd ..

/home/jakob/Bachelorarbeit/GNNDensityGradients


In [2]:

from datasets.density_data_module import DensityDataModule
dm = DensityDataModule(data_dir='datasets/data/dpi_dam_break/train', batch_size=1, data_split=(1, 0, 0), num_workers=0, shuffle=False, cache=True, device='cpu')


In [3]:
dm.setup('fit')
next(iter(dm.train_dataloader()))[0]

Setting up data module for stage  fit
Dataset is already preprepared. Not applying transform_once. (This means density & grads aren't recalculated.)
Loading dataset into memory...


100%|██████████| 11/11 [00:00<00:00, 62.90it/s]

Done loading dataset into memory



/home/jakob/miniconda3/envs/CConv/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jakob/miniconda3/envs/CConv/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


{'pos': tensor([[7.9919e-04, 1.0885e-02, 1.1447e-01],
         [3.4578e-03, 1.0993e-02, 2.2499e-02],
         [3.7652e-03, 1.0935e-02, 9.9544e-02],
         ...,
         [1.4093e+00, 8.0036e-02, 5.0921e-01],
         [1.1419e+00, 1.6014e-01, 6.2688e-01],
         [1.4162e+00, 6.6999e-02, 6.2963e-01]]),
 'vel': tensor([[ 1.5508e-03,  4.7880e-03, -6.5672e-02],
         [ 4.4142e-04,  2.1348e-04, -1.8164e-01],
         [ 3.3536e-04,  2.8863e-03, -7.0346e-02],
         ...,
         [-2.4701e-01,  2.8034e-01, -6.3012e-01],
         [-8.4338e-01, -2.9979e-01,  2.8583e-01],
         [ 1.2954e-01, -5.7610e-01,  2.6140e-01]]),
 'm': tensor([[0.1309],
         [0.1309],
         [0.1309],
         ...,
         [0.1309],
         [0.1309],
         [0.1309]]),
 'viscosity': tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
 'box': tensor([[1.4545e+00, 1.1804e+00, 7.3013e-02],
         [1.4545e+00, 7.3959e-01, 7.0710e-01],
         [1.4545e

In [4]:
next(iter(dm.train_dataloader()))[0].keys()

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals'])